In [77]:
import pandas as pd
import numpy as np
import collections
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.tokenize import word_tokenize

data = pd.read_csv(r'C:\Users\user\Desktop\YEAR 3 SEM 1\TXSA\groupie/Reviews_Sentiments.csv')
data.head()

,Reviews,Sentiment
0,"Not much to write about here, but it does exac...",Positive
1,The product does exactly as it should and is q...,Positive
2,The primary job of this device is to block the...,Positive
3,Nice windscreen protects my MXL mic and preven...,Positive
4,This pop filter is great. It looks and perform...,Positive


In [78]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10254 entries, 0 to 10253
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Reviews    10254 non-null  object
 1   Sentiment  10254 non-null  object
dtypes: object(2)
memory usage: 160.3+ KB


In [79]:
data.Sentiment.value_counts()

Positive    9403
Negative     673
Neutral      178
Name: Sentiment, dtype: int64

In [80]:
def word_features(words):
    return dict([(word, True) for word in words])

#Loops
positive_sentiments = [(neutral['Reviews']) for index, neutral in data.iterrows() if neutral['Sentiment'] == 'Neutral']
neutral_sentiments = [(negative['Reviews']) for index, negative in data.iterrows() if negative['Sentiment'] == 'Negative']
negative_sentiments = [(positive['Reviews']) for index, positive in data.iterrows() if positive['Sentiment'] == 'Positive']


#sentiments and features dictionary are joined.Add counters to all loops.
positive_features = [(word_features(word_tokenize(positive_sentiments[i[0]])),'pos') for i in enumerate(positive_sentiments)]
neutral_features = [(word_features(word_tokenize(neutral_sentiments[i[0]])),'neu') for i in enumerate(neutral_sentiments)]
negative_features = [(word_features(word_tokenize(negative_sentiments[i[0]])),'neg') for i in enumerate(negative_sentiments)]

#Divide dataset into test and train data. 75% for train, 25% for test.
PositiveCutoff = int(len(positive_features)*3/4)
NeutralCutoff = int(len(neutral_features)*3/4)
NegativeCutoff = int(len(negative_features)*3/4)


#testing and trainign featuresets
train_featureset =  positive_features[:PositiveCutoff] + neutral_features[:NeutralCutoff] + negative_features[:NegativeCutoff] 
test_featureset=  positive_features[:PositiveCutoff]+ neutral_features[:NeutralCutoff] + negative_features[:NegativeCutoff] 


#Supervised sentiment classification model using  Naive Bayes classifier with NLTK.
classifier_NB = nltk.NaiveBayesClassifier.train(test_featureset)
refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)

for i, (feats, label) in enumerate(test_featureset):
    refsets[label].add(i)
    observed = classifier_NB.classify(feats)
    testsets[observed].add(i)


print("~~~~~~~~~~~~~~~~~~ PERFORMANCE MEASURES ~~~~~~~~~~~~~~~~~~~~")
print("\nThe accuracy of the classifier is: ", nltk.classify.accuracy(classifier_NB, test_features))
print("\nThe Positive Precision of the classifier is:", nltk.scores.precision(refsets['pos'], testsets['pos']))
print ("\nThe Positive Recall of the classifier is:", nltk.scores.recall(refsets['pos'], testsets['pos']))
print ("\nThe Positive F1 Score of the classifier is:", nltk.scores.f_measure(refsets['pos'], testsets['pos']))

print("\nThe Negative Precision of the classifier is:", nltk.scores.precision(refsets['neg'], testsets['neg']))
print ("\nThe Negative Recall of the classifier is:", nltk.scores.recall(refsets['neg'], testsets['neg']))
print ("\nThe Negative F1 Score of the classifier is:", nltk.scores.f_measure(refsets['neg'], testsets['neg']))



print("~~~~~~~~~~~~~~~~~~ NEUTRALS ~~~~~~~~~~~~~~~~~~~~")
print("\n\nThe Neutral Precision of the classifier is:", nltk.scores.precision(refsets['neu'], testsets['neu']))
print ("\nThe Neutral Recall of the classifier is:", nltk.scores.recall(refsets['neu'], testsets['neu']))
print ("\nThe Neutral F1 Score of the classifier is:", nltk.scores.f_measure(refsets['neu'], testsets['neu']))

~~~~~~~~~~~~~~~~~~ PERFORMANCE MEASURES ~~~~~~~~~~~~~~~~~~~~

The accuracy of the classifier is:  0.9452464559760697

The Positive Precision of the classifier is: 0.3790322580645161

The Positive Recall of the classifier is: 0.3533834586466165

The Positive F1 Score of the classifier is: 0.3657587548638132

The Negative Precision of the classifier is: 0.9624773329613614

The Negative Recall of the classifier is: 0.9784458309699376

The Negative F1 Score of the classifier is: 0.970395893397089
~~~~~~~~~~~~~~~~~~ NEUTRALS ~~~~~~~~~~~~~~~~~~~~


The Neutral Precision of the classifier is: 0.8106060606060606

The Neutral Recall of the classifier is: 0.6369047619047619

The Neutral F1 Score of the classifier is: 0.7133333333333333
